## Human PCLS: cellular crosstalk with NicheNetNiklas after Fibrotic Cocktail

In [1]:
suppressPackageStartupMessages(library(nichenetr))
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(Matrix))
suppressPackageStartupMessages(library(igraph))
suppressPackageStartupMessages(library(gridExtra))
suppressPackageStartupMessages(library(viridis))

In [2]:
## set working directory
setwd('/home/niklas/projects/niche_environments_FIBROSIS/HUMAN_invivo/02_figures/NicheNet/')

### Path to required input data

In [3]:
dge_dir = '/home/niklas/projects/niche_environments_FIBROSIS/IPF_cell_atlas_reference_CPC/01_data/DGE_IPF_vs_healthy/'
table_dir = '/home/niklas/projects/niche_environments_FIBROSIS/HUMAN_invivo/01_data/NicheNet_inputs/'
results_dir = '/home/niklas/projects/niche_environments_FIBROSIS/HUMAN_invivo/01_data/NicheNet_outputs/ligand_activities_FC_CC/'

### Load NicheNet models and networks

In [4]:
## load NicheNet models and networks ##
ligand_target_matrix <- readRDS('/home/niklas/data/nichenet_models/ligand_target_matrix_HUMAN.rds')
lr_network <- readRDS('/home/niklas/data/nichenet_models/ligand_receptor_network_HUMAN.rds')
weighted_networks_lr <- readRDS('/home/niklas/data/nichenet_models/weighted_ligand_receptor_network_HUMAN.rds')

### Function to perform NicheNet Ligand activity analysis

In [5]:
## function to perform ligand activity analysis ##
ligand_activity_analysis <- function(sender_ct, receiver_ct, geneset_oi, 
                            geneset_title,
                            pct_expr_table, pct_thresh = 0.05,
                            pearson_thresh = 0.08){
    
    
    ## retrieve genes expressed by receiver
    expr_genes_receiver = rownames(pct_expr_table[pct_expr_table[, receiver_ct] > pct_thresh, ])
    background_expr_genes = expr_genes_receiver %>% .[. %in% rownames(ligand_target_matrix)]
    
    ## retrieve genes expressed by sender
    list_expr_genes_sender = lapply(sender_ct, function(x){rownames(pct_expr_table[pct_expr_table[, x] > pct_thresh, ])})
    expr_genes_sender = list_expr_genes_sender %>% unlist() %>% unique()
    
    ## status message ##
    print(paste0("Using ", length(geneset_oi), " genes differently regulated genes in ", receiver_ct, " (",
                 geneset_title, ")"))
    
    ### STEP1: Ligand activity analysis ###
    ## Define a set of potential ligands and receptors 
    # retrieve ligands and receptors
    ligands = lr_network %>% pull(from) %>% unique()
    receptors = lr_network %>% pull(to) %>% unique()
    # ligands expressed by sender celltypes
    expr_ligands = intersect(ligands, expr_genes_sender) 
    # receptor expressed by receiver celltypes
    expr_receptors = intersect(receptors, expr_genes_receiver)
    ### status messages ###
    print(paste0("Expressed Ligands ", length(expr_ligands)))
    print(paste0("Expressed Receptors ", length(expr_receptors)))
    
    ## filter ligands
    # only consider ligands with matching receptors (according to NicheNets databases)
    potential_ligands = lr_network %>% filter(from %in% expr_ligands & to %in% expr_receptors) %>%
                        pull(from) %>% unique()
    ### status message ###
    print(paste0("Potential Ligands ", length(potential_ligands)))
    
    ## predict ligand activities
    ligand_activities = predict_ligand_activities(geneset = geneset_oi,
                                                  background_expressed_genes = background_expr_genes,
                                                  ligand_target_matrix = ligand_target_matrix,
                                                  potential_ligands = potential_ligands)

    ## rank ligands by pearson correlation coefficient
    ligand_activities = ligand_activities %>% arrange(-pearson) %>% mutate(rank = rank(desc(pearson)))
    
    # filter consider ligands with pearson's correlation >= pearson_tresh
    ligand_activities = ligand_activities %>% filter(pearson >= pearson_thresh)
    
    ### status message ###
    print(paste0("Top ranked ligands ", length(ligand_activities$test_ligand)))
    
    return(ligand_activities)  
}

In [6]:
target_gene_prediction <- function(best_upstream_ligands,geneset_oi, target_thresh = 0.33, n_targets = 500){
    
    ## identify ligand targets
    active_ligand_target_links_df = best_upstream_ligands %>% 
                                    lapply(get_weighted_ligand_target_links,geneset = geneset_oi, ligand_target_matrix = ligand_target_matrix, n = n_targets) %>% bind_rows() %>% drop_na()
    active_ligand_target_links = prepare_ligand_target_visualization(ligand_target_df = active_ligand_target_links_df, ligand_target_matrix = ligand_target_matrix, cutoff = target_thresh)
    
    ## reformat data
    order_ligands = intersect(best_upstream_ligands, colnames(active_ligand_target_links)) %>% rev() %>% make.names()
    order_targets = active_ligand_target_links_df$target %>% unique() %>% 
                    intersect(rownames(active_ligand_target_links)) %>% make.names()
    rownames(active_ligand_target_links) = rownames(active_ligand_target_links) %>% make.names() 
    colnames(active_ligand_target_links) = colnames(active_ligand_target_links) %>% make.names() # make.names() for heatmap visualization of genes like H2-T23
    
    ## final ligand-target heatmap
    vis_ligand_target = active_ligand_target_links[order_targets,order_ligands] %>% t()
    
    ## output: ligand-target matrix
    return(vis_ligand_target) 
    
}

### Run NicheNet iteratively to detect crosstalk between all cells for FC vs CC

In [7]:
## read avg expression table
avg_expr <- read.csv(paste0(table_dir, '220207_IPF_atlas_invivo_healthy_avg_expr_SCALED.csv'), row.names = 1, check.names = F, header = T)
pct_expr <- read.csv(paste0(table_dir, '220207_IPF_atlas_invivo_healthy_pct_expr.csv'), row.names = 1, check.names = F, header = T)

In [8]:
cell_type_names <- c('Airway Epithelium','Alveolar Epithelium',
                      'capillary EC', 'vascular EC', 'lymphatic EC',
                      'Fibroblasts','SMC/Pericytes',
                      'Monocytes','Macrophages','DC','Mast cells',
                      'B cells','Plasma cells','T cells','NK cells')
cell_type_labels <- c('Airway_Epithelium','Alveolar_Epithelium',
                      'capillary_EC', 'vascular_EC', 'lymphatic_EC',
                      'Fibroblasts','SMC_Pericytes',
                      'Monocytes','Macrophages','DC','Mast_cells',
                      'B_cells','Plasma_cells','T_cells','NK_cells')

In [9]:
## create final results matrix
quant_results_df <- data.frame(matrix(ncol = length(cell_type_names), nrow = length(cell_type_names)))
colnames(quant_results_df) <- cell_type_names # RECEIVER (!!!)
rownames(quant_results_df) <- cell_type_names # SENDER (!!!)
quant_results_df

,Airway Epithelium,Alveolar Epithelium,capillary EC,vascular EC,lymphatic EC,Fibroblasts,SMC/Pericytes,Monocytes,Macrophages,DC,Mast cells,B cells,Plasma cells,T cells,NK cells
,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
Airway Epithelium,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Alveolar Epithelium,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
capillary EC,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
vascular EC,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
lymphatic EC,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Fibroblasts,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
SMC/Pericytes,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Monocytes,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Macrophages,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [10]:
qual_results_df <- copy(quant_results_df)
qual_results_df

,Airway Epithelium,Alveolar Epithelium,capillary EC,vascular EC,lymphatic EC,Fibroblasts,SMC/Pericytes,Monocytes,Macrophages,DC,Mast cells,B cells,Plasma cells,T cells,NK cells
,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
Airway Epithelium,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Alveolar Epithelium,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
capillary EC,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
vascular EC,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
lymphatic EC,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Fibroblasts,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
SMC/Pericytes,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Monocytes,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Macrophages,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


### Run NicheNet

In [11]:
for(i in 1:length(cell_type_names)){
    
    ## step 1: define RECEIVER and SENDER cells
    all_senders <- cell_type_names
    all_sender_labels <- cell_type_labels
    receiver_ct <- cell_type_names[i]
    receiver_labels <- cell_type_labels[i]
    
    ## step 2: define geneset of interest
    # read dge table
    dge_table <- read.csv(paste0(dge_dir, '210616_CPC_IPF_reference_DGE_IPF_vs_healthy_' , cell_type_labels[i], '.csv'))
    # only significant genes: pval_adj < 0.05
    dge_table = dge_table %>% filter(qval < 0.05)
    # only genes expressed in at least 10% per group
    dge_table = dge_table %>% filter(pct.healthy > 0.1)
    dge_table = dge_table %>% filter(pct.ILD > 0.1)
    # only upregulated genes with log2fc > 0.25
    dge_genes_up = dge_table %>% filter(log2fc > 0.25)
    geneset_oi <- dge_genes_up$gene
    geneset_title <- paste0(cell_type_names[i], ' (IPF vs. healthy)')
    print(paste0(cell_type_names[i], ' (IPF vs. healthy): Proceeding with geneset of interest of length: ', length(geneset_oi)))
    
    ## step 3: run ligand activity analysis
    top_ligands_table <- ligand_activity_analysis(all_senders, receiver_ct, geneset_oi = geneset_oi,
                                           geneset_title = geneset_title, pct_thresh = 0.05,
                                           pct_expr_table = pct_expr, pearson_thresh = 0.05)
    
    ## dissect crosstalk between all pairs of cell types quantitatively and qualitatively
    
    # for each sender cell type check which top ligands are upregulated
    for(i in 1:length(all_senders)){
        sender_ct <- all_senders[i]
        sender_label <- all_sender_labels[i]
        # read respective DGE table
        # read dge table
        dge_table <- read.csv(paste0(dge_dir, '210616_CPC_IPF_reference_DGE_IPF_vs_healthy_' , sender_label, '.csv'))
        
        # subset DGE table to top ligands
        ligands_dge = dge_table %>% filter(gene %in% top_ligands_table$test_ligand) 
        
        # filter DGE table by logFC and q-Value
        # if logFC > 0.25 and q-Value < 0.05, a ligand is upregulated
        ligands_dge = ligands_dge %>% filter(qval < 0.05)
        ligands_dge = ligands_dge %>% filter(log2fc > 0.25)
        
        ## add info whether gene is upregulated in SENDER cell type to ligand activity table
        top_ligands_table <- top_ligands_table %>% mutate(new_col = case_when(test_ligand %in% ligands_dge$gene ~ 1,
                                         !(test_ligand %in% ligands_dge$gene) ~ 0))
        colnames(top_ligands_table)[colnames(top_ligands_table) == 'new_col'] <- paste0(sender_label, '_UP')
        
        ### quantitative approach ### 
        # count genes in resulting DGE table --> no. of upregulated top ligands
        ct_quant_score <- length(ligands_dge$gene)
        cat('\n')
        print(paste0(sender_ct, ' --> ', receiver_ct, ' - Top ranked ligands: ', length(top_ligands_table$test_ligand)))
        print(paste0(sender_ct, ' --> ', receiver_ct, ' - Thereof upregulated: ', ct_quant_score))
        cat('\n')
        # fill final results matrix
        quant_results_df[sender_ct, receiver_ct] <- ct_quant_score
        
        ### qualitative approach ### 
        
        # filter ligands results table: upregulated ligands only
        ligands_up = top_ligands_table %>% filter(test_ligand %in% ligands_dge$gene)
        
        # filter avg expression table: only sender cell type
        ligands_avg_expr = avg_expr %>% select(sender_ct)
        # filter avg expression table: only top ligands
        ligands_avg_expr = ligands_avg_expr %>% filter(rownames(ligands_avg_expr) %in% ligands_dge$gene)
        ligands_avg_expr$gene <- rownames(ligands_avg_expr)
        rownames(ligands_avg_expr) <- NULL
        
        # merge ligands results table with ligands avg expression table
        ligands_up <- merge.data.frame(x = ligands_up, y = ligands_avg_expr, by.x = 'test_ligand', by.y = 'gene')
        
        # add new colum to ligand results table: avg expression * ligand score (pearsons rho)
        ligands_up$contribution <- ligands_up[, sender_ct] * ligands_up$pearson
        # sender celltype contribution score equals the sum of the new column
        ct_qual_score <- sum(ligands_up$contribution)
        
        # fill final results matrix
        qual_results_df[sender_ct, receiver_ct] <- ct_qual_score

    }
    
    ## save ligand activity analysis
    print(paste0('Writing ligand activity table to:  ', results_dir, 'nichenet_ligand_act_', receiver_labels, '.csv'))
    write.csv(top_ligands_table, paste0(results_dir, 'nichenet_ligand_act_', receiver_labels, '.csv'))
    
    ## step 4: target gene prediction
    ligand_target_prediction <- target_gene_prediction(best_upstream_ligands = top_ligands_table$test_ligand, geneset_oi = geneset_oi)
    print(paste0('Writing ligand target matrix table to:  ', results_dir, 'nichenet_ligand_target_matrix_', receiver_labels, '.csv'))
    write.csv(ligand_target_prediction, paste0(results_dir, 'nichenet_ligand_target_matrix_', receiver_labels, '.csv'))
}

[1] "Airway Epithelium (IPF vs. healthy): Proceeding with geneset of interest of length: 504"
[1] "Using 504 genes differently regulated genes in Airway Epithelium (Airway Epithelium (IPF vs. healthy))"
[1] "Expressed Ligands 347"
[1] "Expressed Receptors 168"
[1] "Potential Ligands 253"
[1] "Top ranked ligands 0"

[1] "Airway Epithelium --> Airway Epithelium - Top ranked ligands: 0"
[1] "Airway Epithelium --> Airway Epithelium - Thereof upregulated: 0"



Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(sender_ct)` instead of `sender_ct` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.




[1] "Alveolar Epithelium --> Airway Epithelium - Top ranked ligands: 0"
[1] "Alveolar Epithelium --> Airway Epithelium - Thereof upregulated: 0"


[1] "capillary EC --> Airway Epithelium - Top ranked ligands: 0"
[1] "capillary EC --> Airway Epithelium - Thereof upregulated: 0"


[1] "vascular EC --> Airway Epithelium - Top ranked ligands: 0"
[1] "vascular EC --> Airway Epithelium - Thereof upregulated: 0"


[1] "lymphatic EC --> Airway Epithelium - Top ranked ligands: 0"
[1] "lymphatic EC --> Airway Epithelium - Thereof upregulated: 0"


[1] "Fibroblasts --> Airway Epithelium - Top ranked ligands: 0"
[1] "Fibroblasts --> Airway Epithelium - Thereof upregulated: 0"


[1] "SMC/Pericytes --> Airway Epithelium - Top ranked ligands: 0"
[1] "SMC/Pericytes --> Airway Epithelium - Thereof upregulated: 0"


[1] "Monocytes --> Airway Epithelium - Top ranked ligands: 0"
[1] "Monocytes --> Airway Epithelium - Thereof upregulated: 0"


[1] "Macrophages --> Airway Epithelium - Top ranked ligands: 0

Warning message:
“Unknown or uninitialised column: `weight`.”
Warning message:
“Unknown or uninitialised column: `target`.”
Warning message:
“Unknown or uninitialised column: `ligand`.”
Warning message:
“Unknown or uninitialised column: `target`.”
Warning message:
“Unknown or uninitialised column: `ligand`.”
Warning message:
“Unknown or uninitialised column: `target`.”
Warning message:
“Unknown or uninitialised column: `ligand`.”
Warning message:
“Unknown or uninitialised column: `target`.”


[1] "Writing ligand target matrix table to:  /home/niklas/projects/niche_environments_FIBROSIS/HUMAN_invivo/01_data/NicheNet_outputs/ligand_activities_FC_CC/nichenet_ligand_target_matrix_Airway_Epithelium.csv"
[1] "Alveolar Epithelium (IPF vs. healthy): Proceeding with geneset of interest of length: 1081"
[1] "Using 1081 genes differently regulated genes in Alveolar Epithelium (Alveolar Epithelium (IPF vs. healthy))"
[1] "Expressed Ligands 347"
[1] "Expressed Receptors 152"
[1] "Potential Ligands 247"
[1] "Top ranked ligands 239"

[1] "Airway Epithelium --> Alveolar Epithelium - Top ranked ligands: 239"
[1] "Airway Epithelium --> Alveolar Epithelium - Thereof upregulated: 21"


[1] "Alveolar Epithelium --> Alveolar Epithelium - Top ranked ligands: 239"
[1] "Alveolar Epithelium --> Alveolar Epithelium - Thereof upregulated: 50"


[1] "capillary EC --> Alveolar Epithelium - Top ranked ligands: 239"
[1] "capillary EC --> Alveolar Epithelium - Thereof upregulated: 41"


[1] "vascular EC --

Warning message:
“Unknown or uninitialised column: `weight`.”
Warning message:
“Unknown or uninitialised column: `target`.”
Warning message:
“Unknown or uninitialised column: `ligand`.”
Warning message:
“Unknown or uninitialised column: `target`.”
Warning message:
“Unknown or uninitialised column: `ligand`.”
Warning message:
“Unknown or uninitialised column: `target`.”
Warning message:
“Unknown or uninitialised column: `ligand`.”
Warning message:
“Unknown or uninitialised column: `target`.”


[1] "Writing ligand target matrix table to:  /home/niklas/projects/niche_environments_FIBROSIS/HUMAN_invivo/01_data/NicheNet_outputs/ligand_activities_FC_CC/nichenet_ligand_target_matrix_Plasma_cells.csv"
[1] "T cells (IPF vs. healthy): Proceeding with geneset of interest of length: 296"
[1] "Using 296 genes differently regulated genes in T cells (T cells (IPF vs. healthy))"
[1] "Expressed Ligands 347"
[1] "Expressed Receptors 99"
[1] "Potential Ligands 189"
[1] "Top ranked ligands 182"

[1] "Airway Epithelium --> T cells - Top ranked ligands: 182"
[1] "Airway Epithelium --> T cells - Thereof upregulated: 14"


[1] "Alveolar Epithelium --> T cells - Top ranked ligands: 182"
[1] "Alveolar Epithelium --> T cells - Thereof upregulated: 42"


[1] "capillary EC --> T cells - Top ranked ligands: 182"
[1] "capillary EC --> T cells - Thereof upregulated: 30"


[1] "vascular EC --> T cells - Top ranked ligands: 182"
[1] "vascular EC --> T cells - Thereof upregulated: 23"


[1] "lymphatic EC -->

In [12]:
write.csv(quant_results_df, paste0(results_dir, '220207_quant_crosstalk_IPF_healthy.csv'))
quant_results_df

,Airway Epithelium,Alveolar Epithelium,capillary EC,vascular EC,lymphatic EC,Fibroblasts,SMC/Pericytes,Monocytes,Macrophages,DC,Mast cells,B cells,Plasma cells,T cells,NK cells
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Airway Epithelium,0,21,18,20,0,25,22,11,0,0,11,12,0,14,15
Alveolar Epithelium,0,50,46,51,0,56,53,29,0,2,34,31,0,42,41
capillary EC,0,41,38,40,0,46,40,32,0,2,19,28,0,30,29
vascular EC,0,27,24,26,0,29,26,17,0,1,11,13,0,23,21
lymphatic EC,0,15,14,15,0,15,16,10,0,0,5,7,0,10,11
Fibroblasts,0,61,55,57,1,67,58,42,0,4,37,36,0,51,50
SMC/Pericytes,0,50,49,47,0,53,50,33,0,2,33,31,0,45,39
Monocytes,0,17,15,17,0,17,19,13,0,0,9,10,0,14,17
Macrophages,0,22,22,22,0,25,24,21,0,2,17,12,0,20,17


In [13]:
write.csv(qual_results_df, paste0(results_dir, '220207_qual_crosstalk_IPF_healthy.csv'))
qual_results_df

,Airway Epithelium,Alveolar Epithelium,capillary EC,vascular EC,lymphatic EC,Fibroblasts,SMC/Pericytes,Monocytes,Macrophages,DC,Mast cells,B cells,Plasma cells,T cells,NK cells
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Airway Epithelium,0,1.2541061,1.0084967,0.7288352,0.00000000,1.0182716,0.9933199,0.3027240,0.00000000,0.00000000,0.4894918,0.4344750,0,0.6671166,0.6721338
Alveolar Epithelium,0,2.9020258,2.3606030,1.6020304,0.00000000,2.3003470,2.5419929,0.7938084,0.00000000,0.02032307,1.3986479,1.0005040,0,2.0837903,1.6148609
capillary EC,0,2.2262758,1.8116763,1.1621441,0.00000000,1.7619862,1.8262043,0.8929216,0.00000000,0.02581323,0.6542556,0.7512418,0,1.4940101,1.1055799
vascular EC,0,1.5145610,1.1393725,0.7627147,0.00000000,1.2447438,1.1920753,0.6529527,0.00000000,0.06642884,0.4217670,0.4444174,0,1.1739118,0.8390571
lymphatic EC,0,0.9740630,0.7995502,0.5556148,0.00000000,0.7590037,0.9059968,0.4328953,0.00000000,0.00000000,0.3042244,0.3450289,0,0.7132733,0.6067644
Fibroblasts,0,4.0571411,3.2959720,2.1584393,0.05090826,3.4225446,3.2365885,1.6134819,0.00000000,0.15408308,1.9787909,1.4012248,0,3.2174505,2.4536372
SMC/Pericytes,0,2.2238301,2.1167650,1.1523576,0.00000000,1.8852263,1.8877700,0.9046159,0.00000000,0.02216873,0.9087499,0.7728042,0,1.8317527,1.2082657
Monocytes,0,1.3258681,1.0706921,0.7335577,0.00000000,1.0442990,1.4068405,0.6799049,0.00000000,0.00000000,0.5108892,0.3162811,0,0.9296119,0.9139480
Macrophages,0,1.5022383,1.3045644,0.8080147,0.00000000,1.2543075,1.3785651,0.6263757,0.00000000,0.01725455,0.7779727,0.3641936,0,1.0823083,0.8093529
